In [ ]:
import os
import shutil
import numpy as np

In [ ]:
fol = ['test', 'train', 'valid']

base = 'C:/Users/benjaminli/documents/coding/webscrape-inaturalist/'
final = 'C:/Users/benjaminli/documents/coding/webscrape-inaturalist/combined_ds'

for f in fol:
    for img in os.listdir(f'C:/Users/benjaminli/documents/coding/webscrape-inaturalist/b_ds/{f}/images'):
        shutil.copy(f'C:/Users/benjaminli/documents/coding/webscrape-inaturalist/b_ds/{f}/images/{img}', f'{final}/images/{img}')
    for label in os.listdir(f'b_ds/{f}/labels'):
        shutil.copy(f'C:/Users/benjaminli/documents/coding/webscrape-inaturalist/b_ds/{f}/labels/{label}', f'{final}/labels/{label}')

In [ ]:
path = os.getcwd()
names = ['anabaena', 'aphanizomenon', 'detritus', 'dolichospermum', 'microcystis', 'oscillatoria', 'synechococcus', 'water bubble', 'woronichinia']
names_with_freqs = [0 for i in range(len(names))]
x=0
for label in os.listdir(path + "/combined_ds/labels"):
    # organize everything into folders based on
    
    with open(path + f"/combined_ds/labels/{label}") as file:
        # read first line
        asdf = file.readline().split(" ")[0]
        print(asdf)
        try:
            numval = int(asdf)
            print(numval)
            real_name = names[numval]
            # create folder if it doesn't exist
            if not os.path.exists(path + f"/org_ds/{real_name}"):
                os.mkdir(path + f"/org_ds/{real_name}")
            # move image and label to folder
            shutil.copy(path + f"/combined_ds/images/{label[:-4]}.jpg", path + f"/org_ds/{real_name}/{f'{real_name}_{names_with_freqs[numval]+1}'}.jpg")
            x+=1
            names_with_freqs[numval] += 1
            # copy the label file to labels folder
            shutil.copy(path + f"/combined_ds/labels/{label}", path + f"/org_ds/labels/{f'{real_name}_{names_with_freqs[numval]}'}.txt")
        except:
            print("failure")
            # label does not exist
print(x)

In [ ]:
names = ['anabaena', 'aphanizomenon', 'detritus', 'dolichospermum', 'microcystis', 'oscillatoria', 'synechococcus', 'water bubble', 'woronichinia']
path = os.getcwd()

ftrain, fval, ftest = np.array([]), np.array([]), np.array([])
# stratify splitting of data
for name in names:
    allFileNames = os.listdir(path + f"/org_ds/{name}")
    print(len(allFileNames))

    np.random.shuffle(allFileNames)

    train, val, test = np.split(np.array(allFileNames),[int(len(allFileNames)*0.8), int(len(allFileNames)*0.9)])
    print(len(train), len(val), len(test))

    ftrain = np.concatenate((ftrain, train))
    fval = np.concatenate((fval, val))
    ftest = np.concatenate((ftest, test))

In [ ]:
print(len(ftrain), len(fval), len(ftest))

In [ ]:
names = ['anabaena', 'aphanizomenon', 'detritus', 'dolichospermum', 'microcystis', 'oscillatoria', 'synechococcus', 'water bubble', 'woronichinia']
name_counter = [0 for i in range(len(names))]
for f in ftrain:
    for n in names:
        if n in f:
            name_counter[names.index(n)] += 1

asdfasdf = 0
for name in range(len(names)):
    print(names[name], name_counter[name])
    asdfasdf += name_counter[name]
print(asdfasdf)

In [ ]:
# re-organize everything back into same format

splits = ['test', 'train', 'valid']
if not os.path.exists(path + "/final_ds"):
    os.mkdir(path + "/final_ds")

for split in splits:
    if not os.path.exists(path + f"/final_ds/{split}"):
        os.mkdir(path + f"/final_ds/{split}")    
    if not os.path.exists(path + f"/final_ds/{split}/images"):
        os.mkdir(path + f"/final_ds/{split}/images")
    if not os.path.exists(path + f"/final_ds/{split}/labels"):
        os.mkdir(path + f"/final_ds/{split}/labels")
    
    thing = ftrain if split == 'train' else fval if split == 'val' else ftest
    print(thing)
    for file in thing:
        c = file.split("_")[0]
        shutil.copy(path + f"/org_ds/{c}/{file}", path + f"/final_ds/{split}/images/{file}")
        shutil.copy(path + f"/org_ds/labels/{file[:-4]}.txt", path + f"/final_ds/{split}/labels/{file[:-4]}.txt")

In [ ]:
# add data.yaml file
with open(path + "/final_ds/data.yaml", "w") as file:
    file.write("train: final_ds/train/images")
    file.write("val: final_ds/val/images")
    file.write("nc: 9")
    file.write("names: ['anabaena', 'aphanizomenon', 'detritus', 'dolichospermum', 'microcystis', 'oscillatoria', 'synechococcus', 'water bubble', 'woronichinia']")

In [ ]:
print(len(os.listdir(path + "/final_ds/train/images")))

In [ ]:
def rebalance_dataset():
    pass